In [17]:
import pandas as pd

import bs4
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [18]:
driver = webdriver.Chrome()
driver.get('https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023')

In [19]:
source = bs4.BeautifulSoup(driver.page_source)


GAME_DATA = { 'name': [], 'team': [], 'min': [], 'goal': [], 'assist': [], 'shot': [], 'pass': [], 'aer': [], 'mom': [], 'rating': [] }
SEASON = source.find('select', attrs={'id': 'seasons'}).find('option', attrs={"selected": "selected"}).string.replace('/','-')

In [20]:
driver.find_elements(By.XPATH, '//a[@data-backbone-model-attribute="isMinApp"]')[-1].click()
driver.execute_script("window.scrollBy(0,300);")

source = bs4.BeautifulSoup(driver.page_source)


In [21]:
def save_page(source):
    players = source.find_all('a', attrs={'class': 'player-link'})
    player_names = []
    for i in range(0, len(players), 2):
        player_names.append(' '.join(players[i]['href'].split('/')[-1].split('-')))
    GAME_DATA['name'] += player_names[:-20]

    teams = source.find_all('span', attrs={'class': 'team-name'})
    team_names = []
    for i in range(0, len(teams), 2):
        team_names.append(teams[i].string.split(',')[0])
    GAME_DATA['team'] += team_names

    minsPlayed = source.find_all('td', attrs={'class': 'minsPlayed'})
    mins_played = []
    for i in range(0, len(minsPlayed), 1):
        mins_played.append(0 if minsPlayed[i].string.strip() == '-' else int(minsPlayed[i].string))
    GAME_DATA['min'] += mins_played

    goals = source.find_all('td', attrs={'class': 'goal'})
    goals_scored = []
    for i in range(0, len(goals), 1):
        goals_scored.append(0 if goals[i].string.strip() == '-' else int(goals[i].string))
    GAME_DATA['goal'] += goals_scored

    assits = source.find_all('td', attrs={'class': 'assistTotal'})
    goals_assisted = []
    for i in range(0, len(assits), 1):
        goals_assisted.append(0 if assits[i].string.strip() == '-' else int(assits[i].string))
    GAME_DATA['assist'] += goals_assisted

    shots = source.find_all('td', attrs={'class': 'shotsPerGame'})
    shots_per_game = []
    for i in range(0, len(shots), 1):
        shots_per_game.append(0 if shots[i].string.strip() == '-' else float(shots[i].string))
    GAME_DATA['shot'] += shots_per_game

    passes = source.find_all('td', attrs={'class': 'passSuccess'})
    pass_success = []
    for i in range(0, len(passes), 1):
        pass_success.append(0 if passes[i].string.strip() == '-' else float(passes[i].string)/100)
    GAME_DATA['pass'] += pass_success

    aers = source.find_all('td', attrs={'class': 'aerialWonPerGame'})
    aer_wins = []
    for i in range(0, len(aers), 1):
        aer_wins.append(0 if aers[i].string.strip() == '-' else float(aers[i].string))
    GAME_DATA['aer'] += aer_wins

    moms = source.find_all('td', attrs={'class': 'manOfTheMatch'})
    mom = []
    for i in range(0, len(moms), 1):
        mom.append(0 if moms[i].string.strip() == '-' else int(moms[i].string))
    GAME_DATA['mom'] += mom

    ratings = source.find_all('td', attrs={'class': 'rating'})
    game_rating = []
    for i in range(0, len(ratings), 1):
        game_rating.append(0 if ratings[i].string.strip() == '-' else float(ratings[i].string))
    GAME_DATA['rating'] += game_rating

In [22]:
save_page(source)
# for column in GAME_DATA:
#     print(column, len(GAME_DATA[column]))
# GAME_DATA['name']

In [23]:
while source.find('a', attrs={'id': 'next', 'class': 'clickable'}):
    # WebDriverWait(driver, 10).until(
    #     EC.presence_of_element_located((By.XPATH, "//a[@id='next']"))
    # ).click()
    # driver.implicitly_wait(3)
    time.sleep(0.5)
    driver.find_element(By.XPATH, "//a[@id='next']").click()
    source = bs4.BeautifulSoup(driver.page_source)
    save_page(source)

pd.DataFrame(GAME_DATA).to_csv(f'seasons/England-Premier-League/{SEASON}.csv')

In [24]:
for title in GAME_DATA:
    print(title, len(GAME_DATA[title]))

name 579
team 579
min 579
goal 579
assist 579
shot 579
pass 579
aer 579
mom 579
rating 579


In [25]:
driver.close()